You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(  # descriptipn描述了任务是什么，agent要怎么做 
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence and its impact on various industries
- Importance of staying updated on the latest trends, key players, and news in the AI sector

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing developments
- AI ethics and bias considerations
- Autonomous vehicles and robotics
- Personalization and recommendation systems

2.

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work, communicate, and live. From machine learning advancements to natural language processing developments, AI has the potential to shape the future in unprecedented ways. Staying updated on the latest trends, key players, and news in the AI sector is crucial for understanding the impact of this technology on society and the economy.

## Latest Trends in Artificial Intelligence

In the field of AI, machine learning advancements continue to drive innovation, enabling computers to learn from data and improve their performance without being explicitly programmed. Natural language processing developments have also gained momentum, with AI systems becoming more adept at understanding and generating human language. However, as AI technologies evolv

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work, communicate, and live. From machine learning advancements to natural language processing developments, AI has the potential to shape the future in unprecedented ways. Staying updated on the latest trends, key players, and news in the AI sector is crucial for understanding the impact of this technology on society and the economy.

## Latest Trends in Artificial Intelligence

In the field of AI, machine learning advancements continue to drive innovation, enabling computers to learn from data and improve their performance without being explicitly programmed. Natural language processing developments have also gained momentum, with AI systems becoming more adept at understanding and generating human language. However, as AI technologies evolve, ethical considerations and bias issues have come to the forefront, raising important questions about the responsible use of AI. Autonomous vehicles and robotics are also at the forefront of AI innovation, with companies like Tesla pushing the boundaries of self-driving technology. Personalization and recommendation systems powered by AI algorithms are transforming the way we interact with online platforms, creating tailored experiences for users.

## Key Players in the AI Industry

The AI industry is dominated by major players such as Google, Amazon, Microsoft, IBM, Tesla, OpenAI, and DeepMind. These companies are at the forefront of AI research and development, driving innovation and shaping the future of technology. Google's DeepMind, for example, has made significant strides in AI research, particularly in the field of reinforcement learning. Amazon's AWS AI services are widely used across industries, providing scalable and cost-effective AI solutions. Microsoft's Azure AI platform offers a comprehensive suite of AI tools for businesses, while IBM's Watson AI technology is renowned for its cognitive computing capabilities. Tesla's self-driving technology and OpenAI's advancements in AI ethics are also noteworthy contributions to the industry.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research, such as OpenAI's GPT-3 language model and DeepMind's AlphaFold protein folding algorithm, have garnered widespread attention for their potential applications across various sectors. AI is increasingly being deployed in healthcare, finance, and other industries to improve efficiency and decision-making processes. However, ethical concerns and regulations surrounding AI technology remain a key focus area, with policymakers and industry leaders working to address issues of bias, privacy, and transparency. The impact of AI on the future job market is also a topic of debate, with concerns about automation displacing human workers and the need for upskilling and reskilling programs to adapt to a changing workforce.

## Conclusion

As AI continues to advance at a rapid pace, staying informed about the latest trends, key players, and news in the industry is essential for anyone interested in the future of technology. From machine learning and natural language processing to ethical considerations and autonomous vehicles, AI is reshaping the way we interact with the world around us. By keeping up with the latest developments and understanding the implications of AI technology, we can navigate the complex landscape of artificial intelligence with greater clarity and insight.

Remember to subscribe to our newsletter for regular updates on AI trends and news, attend upcoming AI conferences or webinars for in-depth insights, and share this article with your network to spread awareness about the importance of staying informed on AI advancements. Together, we can shape the future of artificial intelligence and harness its potential for the benefit of society and the economy.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).